In [29]:
import pandas as pd
import os
import sys
import argparse
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
from src.utils import *

In [63]:
path = "test/dialogue/t5/gen_test/results"
file = "dataset_measurements.pkl"
measurements = load_pickle(path, file)
# prettyprint_dict(measurements)

In [64]:
metrics = measurements['metrics']
metric_keys = list(metrics.keys())

In [65]:
idx = 9
key = metric_keys[idx]
print(key)
metrics[key]

formality_mean


0.5808199167251586

In [66]:
for k, v in metrics.items():
    print(k, v)

samples 0    Fabienne: i'm so happy to hear it has arrived....
1    Julia: I have a new boyfriend Darline: I’m exh...
2    saturday is a day for eating and drinking. sat...
3    Jamie: I'm down for Saturday too Nicki: I can ...
4    Charles: We're on Elm Street, where now? Charl...
5    Sue: I might be watching the game tonight. Sue...
6    Ellis: "this is my fucking problem" Sara: "I h...
7    Miranda: "you could have asked me if i'm ok wi...
8    Xander: ok, ok, ok, ok, ok, ok, ok, ok, ok, ok...
9    tryce: i'm not Victor: don't think i've not no...
Name: summary, dtype: object
sample_idxs [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
coverage 0.3269883431123983
compression_ratio 0.5475536668441047
summary_token_distribution <FreqDist with 502 samples and 1401 outcomes>
formality_scores [0.         0.         0.99562919 0.99267548 0.60214835 0.60695475
 0.5502696  0.55923361 0.67359525 0.82769293]
formality_sample_idxs [23 17 12 20 31  6  1 29 40 27]
toxicity_scores [0.07054565 0.00785379 0.0262637

In [97]:
prettyprint_dict(measurements)

config
	subject
	generation
	no_samples
	DEBUG
	model
metrics
	samples
	sample_idxs
	coverage
	compression_ratio
	summary_token_distribution
	formality_scores
	formality_sample_idxs
	toxicity_scores
	toxicity_sample_idxs
	formality_mean
	toxicity_mean
	emotion_intensity_mean
	emotion_intensity_measurements
generation
subject


In [113]:
initial_dataset_paths = {
    'dialogue': 'Data/initial_datasets/dialogue/full_data.csv'
}
import copy
from functools import reduce

def generate_model_name_string(config_d):
    return f"{config_d['model']}_gen{config_d['generation']}_{config_d['subject']}"

def generate_summary_comparison(list_res_dicts):
    summary_df_dict = {}
    subjects = []
    for model in list_res_dicts:
        model_name = generate_model_name_string(model['config'])
        subjects.append(model['config']['subject'])
        model_df = {
            'id':model['metrics']['sample_idxs'],
            'summary':model['metrics']['samples'],
        }
        model_df = pd.DataFrame(model_df)
        model_df = model_df.set_index('id')
        summary_df_dict[model_name] = model_df
    if len(set(subjects)) == 1:
        documents = pd.read_csv(initial_dataset_paths[subjects[0]], usecols=['id', 'summary'], index_col=['id'])
        display(documents.sort_index)
    summary_comparison_dfs = pd.concat(summary_df_dict, axis=1)
    return summary_comparison_dfs

        

fake_res = copy.deepcopy(measurements)
fake_res['config']['generation'] = 'FAKE'
fake_res2 = copy.deepcopy(measurements)
fake_res2['config']['generation'] = 'XxXxXx'
test_list = [measurements, fake_res, fake_res2]
test_summary_comp = generate_summary_comparison(test_list)
test_summary_comp

<bound method DataFrame.sort_index of                                                       summary
id                                                           
13819259    Ann, Lee and Max are attending Kate's wedding....
13612091    John and Fiona are at the river. Fiona is alwa...
13730177                  Gemma and Dina are meeting at 6.30.
13830135    Celine and Mark went skating, it's the first t...
13681838-1  Marylin didn't buy a new dress in the mall yes...
...                                                       ...
13730697    Tara's mom bought a new robe so Tara and Ken n...
13862997    Diane is fed up with her job and is looking fo...
13728250    Tom and Ana need to be in the office at 3 pm. ...
13730965-1  The list of bills which is outside the departm...
13865049    Omer got the job. He will be working for Donal...

[16369 rows x 1 columns]>

,t5_gen0_dialogue,t5_genFAKE_dialogue,t5_genXxXxXx_dialogue
,summary,summary,summary
id,,,
0,Fabienne: i'm so happy to hear it has arrived....,Fabienne: i'm so happy to hear it has arrived....,Fabienne: i'm so happy to hear it has arrived....
1,Julia: I have a new boyfriend Darline: I’m exh...,Julia: I have a new boyfriend Darline: I’m exh...,Julia: I have a new boyfriend Darline: I’m exh...
2,saturday is a day for eating and drinking. sat...,saturday is a day for eating and drinking. sat...,saturday is a day for eating and drinking. sat...
3,Jamie: I'm down for Saturday too Nicki: I can ...,Jamie: I'm down for Saturday too Nicki: I can ...,Jamie: I'm down for Saturday too Nicki: I can ...
4,"Charles: We're on Elm Street, where now? Charl...","Charles: We're on Elm Street, where now? Charl...","Charles: We're on Elm Street, where now? Charl..."
5,Sue: I might be watching the game tonight. Sue...,Sue: I might be watching the game tonight. Sue...,Sue: I might be watching the game tonight. Sue...
6,"Ellis: ""this is my fucking problem"" Sara: ""I h...","Ellis: ""this is my fucking problem"" Sara: ""I h...","Ellis: ""this is my fucking problem"" Sara: ""I h..."
7,"Miranda: ""you could have asked me if i'm ok wi...","Miranda: ""you could have asked me if i'm ok wi...","Miranda: ""you could have asked me if i'm ok wi..."


In [120]:
pd.melt(test_summary_comp)

,variable_0,variable_1,value
0,t5_gen0_dialogue,summary,Fabienne: i'm so happy to hear it has arrived....
1,t5_gen0_dialogue,summary,Julia: I have a new boyfriend Darline: I’m exh...
2,t5_gen0_dialogue,summary,saturday is a day for eating and drinking. sat...
3,t5_gen0_dialogue,summary,Jamie: I'm down for Saturday too Nicki: I can ...
4,t5_gen0_dialogue,summary,"Charles: We're on Elm Street, where now? Charl..."
5,t5_gen0_dialogue,summary,Sue: I might be watching the game tonight. Sue...
6,t5_gen0_dialogue,summary,"Ellis: ""this is my fucking problem"" Sara: ""I h..."
7,t5_gen0_dialogue,summary,"Miranda: ""you could have asked me if i'm ok wi..."
8,t5_gen0_dialogue,summary,"Xander: ok, ok, ok, ok, ok, ok, ok, ok, ok, ok..."
9,t5_gen0_dialogue,summary,tryce: i'm not Victor: don't think i've not no...


In [119]:
for i in test_summary_comp.index:
    print(test_summary_comp['t5_gen0_dialogue'].iloc[i].values)

["Fabienne: i'm so happy to hear it has arrived. fabienne: i'll let you know if the parcel gets delivered."]
['Julia: I have a new boyfriend Darline: I’m exhausted Julia: I’m on standby also Julia: I’m exhausted.']
['saturday is a day for eating and drinking. saturday is the first day of the saturday. saturday is the first day of the saturday.']
["Jamie: I'm down for Saturday too Nicki: I can take everyone to Walmart this Saturday!"]
["Charles: We're on Elm Street, where now? Charles: Turn left to Brick Lane and then go straight ahead. Charles: We'll be there soon!"]
["Sue: I might be watching the game tonight. Sue: i've got my own Fan Zone. Sue: i need to be at work early in the morning."]
['Ellis: "this is my fucking problem" Sara: "I have a project to finish" Sara: "I have a project to finish"']
['Miranda: "you could have asked me if i\'m ok with it" "we just went to the movies, no biggie," says Lindsey. Miranda: "i can\'t betray his trust"']
['Xander: ok, ok, ok, ok, ok, ok, ok, ok

In [ ]:
documents = pd.read_csv(initial_dataset_paths[subjects[0]], usecols=['id', 'document'])

In [3]:
# df_train = dialogue_df.iloc[:200]
# df_validate = dialogue_df.iloc[200:250]

In [2]:
# df_train.to_csv('Data/dialogue_train_small.csv', index=False)
# df_validate.to_csv('Data/dialogue_validation_small.csv', index=False)

In [3]:
# df_train.iloc[1]['document']

In [4]:
# config = load_pickle('test/t5/genX/results', 'config.pkl')
# results = load_pickle('test/t5/genX/results', 'results.pkl')

In [5]:
# config

In [6]:
# prettyprint_dict(results)

In [7]:
# results['metrics']['samples']

In [8]:
# results['metrics']['emotion_intensity_measurements']